We import the ICD codes and the dataset-specific codes, then encode them into embeddings using a transformer model. To visualize potential clusters to aid cleaning, we apply UMAP for dimensionality reduction and HDBSCAN for cluster detection.


In [62]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from transformers import AutoTokenizer, AutoModel
from tqdm.auto import tqdm
import re
from itertools import chain
from sklearn.metrics.pairwise import cosine_similarity
import glob

icd_codes = pd.read_csv("../data/raw/all_icd_codes.csv", dtype = str)
dataset_codes = pd.read_csv("../data/raw/dataset_codes.csv", dtype = str)

In [63]:
# Generate list of unique codes and their row origin index
codes = list(set(dataset_codes.code))
ref_codes = list(set(icd_codes.code))

flattened_codes = []
row_ids = []

 # We could look at a histogram
for row_id, code in enumerate(codes):
    split_codes = re.split(r"[,&/\\\s]+", code)
    for fc in split_codes:
        if fc not in flattened_codes:
            flattened_codes.append(fc)
            row_ids.append(row_id)
            
# Filter out empty strings
ref_codes = [rc for rc in ref_codes if rc]
flattened_codes = [fc for fc in flattened_codes if fc]

In [64]:
# Find length of all the unique codes
data = np.array([len(x) for x in flattened_codes if x])

# Create a histogram
# plt.hist(data,
#          bins = 5,
        #  edgecolor='black')
# plt.show()
# All are 3-8 characters long after cleaning

In [65]:
# ONLY 206 codes do not have perfect matches!
# Update: with additional ICD dictionaries, ONLY 9 codes do not have perfect matches!
flattened_set = set(flattened_codes)
ref_set = set(ref_codes)

matched_codes = list(flattened_set & ref_set)
# print(len(flattened_set) - len(matched_codes))

For the 205 codes, we will use embeddings and cosine similarity to help us match them <br> 
Nevermind! For the 9 codes, I'll just do stuff manually. <br>
Nevermind, we can just remove the scientific notation codes from the code list, clean them, reinsert them, and then find the set difference and manually annotate (e14.1 = Diabetes mellitus with ketoacidosis)

In [66]:
unmatched_codes = list(flattened_set.difference(ref_set))
flattened_codes = [i for i in flattened_codes if i not in unmatched_codes]

unmatched_codes = [re.sub(r"\+", "", re.sub(r"(\.[^a-z]*)(?=e)", "", str(x))) for x in unmatched_codes]
flattened_codes += unmatched_codes

In [67]:

flattened_set = set(flattened_codes)
ref_set = set(ref_codes)
matched_codes = list(flattened_set & ref_set)

print(len(flattened_set) - len(matched_codes))
unmatched_codes = list(flattened_set.difference(ref_set))

print(unmatched_codes)


1
['e14.1']


In [68]:
# Annotate - Now we can map all our codes
icd_codes.loc[len(icd_codes)] = ["e14.1", "Diabetes mellitus with ketoacidosis"] # https://icd.who.int/browse10/2019/en#/E10-E14
ref_set = list(set(icd_codes.code)) # For the comprehensive list

In [ ]:
# icd_codes.loc[icd_codes.code == "e14.1", :] # It's in

# Create columns for corrections
dataset_codes["correction_1"] = ""
dataset_codes["correction_2"] = ""
dataset_codes["correction_3"] = ""
dataset_codes["correction_4"] = ""
dataset_codes["correction_5"] = ""
dataset_codes["correction_6"] = ""
dataset_codes["correction_7"] = ""
dataset_codes["correction_8"] = ""

,code,correction_1,correction_2,correction_3,correction_4,correction_5,correction_6,correction_7,correction_8
0,e14.1,,,,,,,,
1,i95.9,,,,,,,,
2,c53.9,,,,,,,,
3,d64.9,,,,,,,,
4,a41.9,,,,,,,,
...,...,...,...,...,...,...,...,...,...
2064,ja00.36,,,,,,,,
2065,8b60.z,,,,,,,,
2066,nc7y,,,,,,,,
2067,bb4y,,,,,,,,


Great! Now we can "back-propagate" our corrections.

In [ ]:
# Inefficient but not too slow. We compile and export the mapping dictionary.
dataset_codes_2 = dataset_codes.copy()

for i, r in dataset_codes_2.iterrows():
    r_code = r.iloc[0]
    r_splits = re.split(r"[,&/\\\s]+", r_code)

    for j, split in enumerate(r_splits):
        if split: 
            if split in ref_set:
                # print(split)
                dataset_codes_2.loc[i, f'correction_{j+1}'] = split

            elif split not in ref_set:
                non_science_split = re.sub(r"\+", "", re.sub(r"(\.[^a-z]*)(?=e)", "", str(split))) 
                if non_science_split in ref_set:
                    dataset_codes_2.loc[i, f'correction_{j+1}'] = non_science_split
                else:
                    print("Uh oh. No non-scientific-format match found:")
                    print(i)
                    print(split)
                    print()
            else:
                print("Uh oh. No match found:")
                print(i)
                print(split)
                print()

In [81]:
dataset_codes_2.to_csv("../data/processed/code_to_code_dictionary.csv", index=False)

We have taken the OG strings, split them, and stored them in a dictionary; OG string: [splits]. Next, we will compare using embeddings (from SapBERT) and cosine similarity. Finally, we will try and reverse-match our OG strings to our dictionary of {reference : now_matched_code}. If we find no match, we split and compare each value in the split to the dictionary.


**Note** I actually didn't run the below; in the future, run it if you need to, save, and use those new (updated) reference embeddings.

In [ ]:
# From https://huggingface.co/cambridgeltl/SapBERT-from-PubMedBERT-fulltext
tokenizer = AutoTokenizer.from_pretrained("cambridgeltl/SapBERT-from-PubMedBERT-fulltext")  
model = AutoModel.from_pretrained("cambridgeltl/SapBERT-from-PubMedBERT-fulltext").cuda()

bs = 128 # batch size during inference
unmatched_embeddings = []
reference_embeddings = []

for i in tqdm(np.arange(0, len(unmatched_codes), bs)):
    toks = tokenizer.batch_encode_plus(unmatched_codes[i:i+bs], 
                                       padding="max_length", 
                                       max_length=25, 
                                       truncation=True,
                                       return_tensors="pt")
    toks_cuda = {}
    for k,v in toks.items():
        toks_cuda[k] = v.cuda()
    cls_rep = model(**toks_cuda)[0][:,0,:] # use CLS representation as the embedding
    unmatched_embeddings.append(cls_rep.cpu().detach().numpy())

for i in tqdm(np.arange(0, len(ref_codes), bs)):
    toks = tokenizer.batch_encode_plus(ref_codes[i:i+bs], 
                                       padding="max_length", 
                                       max_length=25, 
                                       truncation=True,
                                       return_tensors="pt")
    toks_cuda = {}
    for k,v in toks.items():
        toks_cuda[k] = v.cuda()
    cls_rep = model(**toks_cuda)[0][:,0,:] # use CLS representation as the embedding
    reference_embeddings.append(cls_rep.cpu().detach().numpy())

unmatched_embeddings = np.concatenate(unmatched_embeddings, axis=0)
reference_embeddings = np.concatenate(reference_embeddings, axis=0)

In [ ]:
np.save("embeddings/unmatched_embeddings.npy", unmatched_embeddings)
np.save("embeddings/reference_embeddings.npy", reference_embeddings)

# # Load dataset code embeddings
# unmatched_file = sorted(glob.glob("embeddings/unmatched_embeddings.npy"))
# unmatched_embeddings = np.vstack([np.load(f) for f in unmatched_file])

# # Load reference code embeddings
# references_file = sorted(glob.glob("embeddings/reference_embeddings.npy"))
# reference_embeddings = np.vstack([np.load(f) for f in references_file])


Now, we use cosine similarity and edit distance to compare the codes


In [ ]:
# Compute cosine similarity matrix
sim_matrix = cosine_similarity(unmatched_embeddings, reference_embeddings)

# Get top-3 nearest reference codes for each dataset code
top_k = 3
top_matches = np.argsort(-sim_matrix, axis=1)[:, :top_k]

for i, idxs in enumerate(top_matches):
    print(f"{row_ids[i]}: {unmatched_codes[i]} → {[ref_codes[j] for j in idxs]}")

Note that for rows that end up with multiple codes, we will select the code it posseses that shows up with the highest frequency in the database.